> In assignment 3, we used a vanilla transformer for Spanish-English Neural Machine Translation. It took quite a long time to train (compared to those smaller models), so we will learn to use a profiler to identify what exactly is the bottlenecks and possible improvements.
>
> The first part is a quick demo on how to use the TensorFlow Profiler. A couple of post readings will guide you towards different profiler use cases. Then you need to complete the modification to the `Transformer` Class in order to use `.fit()` method. This is required for profiler callbacks during training. After profiling the vanilla transformer, you will describe your profiling results to identify bottlenecks, propose and experiment to reduce or eliminate 3 of these bottlenecks. Discuss your experiment and results. Finally, rerun the non-improved version on ThetaGPU and compare profiling differences, and answer some questions. 
>
> The notebook is at: https://drive.google.com/file/d/1MwaOPAW8xfadGhFlsuziLRf80YfGcOnq/view?usp=sharing


# Experiments & Write up



### 1. Describe your profiling results on Colab, according to your understanding of this particular transformer architecture design. 




As expected (see the screenshot pie charts below for `ColabBase[line]`), much of the time was spent on matrix multiplication as the transformer models not only consist of a lot of `matmul` in the rudimentary components (like `dense`) but especially in the attention component. Additionally, for the overview statistics, only very little time was spent on input as there was not much to be done or transformed for the inputs. Around 10% was spent on `Kernel launch time`, one of the places could have been in the `scaled_dot_product_attention` function. On average, 149.4ms / 175.4ms was the `Device compute time` (around 85.1%), which is understandable given the overwhelming number of `matmul` operations. Following `matmul` ops is `transpose`, which is also understandable as that operation appears quite a lot of times in `MultiheadAttention`: each call performs at least 4 operations.  

In [ ]:
#@title Overview summary 
import yaml
import pandas as pd
results = yaml.safe_load(open('data/mean_time.yml'))
pd.DataFrame(results)


,ThetaGPUBaseline,ColabBaseline,ColabFlagNCache,ColabMinCast,ColabXLA
Average Step Time,71.7 ms,175.4 ms,177.0 ms,175.9 ms,176.1 ms
All Others Time,1.4 ms,4.1 ms,3.9 ms,3.9 ms,4.3 ms
Compilation Time,0.0 ms,0.0 ms,0.0 ms,0.0 ms,0.0 ms
Output Time,0.0 ms,0.0 ms,0.0 ms,0.0 ms,0.0 ms
Input Time,0.1 ms,0.2 ms,0.4 ms,0.6 ms,0.2 ms
Kernel Launch Time,47.0 ms,18.2 ms,18.5 ms,17.1 ms,18.2 ms
Host Compute Time,0.9 ms,3.5 ms,4.1 ms,4.5 ms,3.8 ms
Device Collective Communication Time,0.0 ms,0.0 ms,0.0 ms,0.0 ms,0.0 ms
Device to Device Time,0.0 ms,0.0 ms,0.0 ms,0.0 ms,0.0 ms
Device Compute Time,22.2 ms,149.4 ms,150.0 ms,149.8 ms,149.6 ms


Here's the aggregation of screenshots with the **tensorflow statistics** for different operations.

![screen shot](https://raw.githubusercontent.com/tuanpham96/DeepLearningSystems-Fall2021/main/HW5/media/screenshot-tboard-tfstats.png)

### 2 + 3. Identify bottlenecks in the training phase using these diagrams. 
Choose three bottlenecks you think could be improved or eliminated by employing techniques learned from [Debugging and Optimization](https://colab.research.google.com/drive/1MwaOPAW8xfadGhFlsuziLRf80YfGcOnq?authuser=1#scrollTo=9VEjIwHO8Tv9) readings. 

Carry out experiments (on Colab) testing your modifications to the pipeline / model, discuss:
- Why do you think it is an improvable bottleneck?
- How do you plan to modify to make it better?
- Does your plan work out well? If not, what could be the preventive factor?


#### Try 1 - **`ColabFlagNCache`**: Set `TF_GPU_THREAD_MODE=gpu_private` and use `cache` for data

As the recommendations from Tensorboard say that there's not much input issue but suggest changing `TF_GPU_THREAD_MODE` to reduce `Kernel launch time`, the first thing I did was to apply this change and also add `cache` for loading the dataset to maximize [recommended optimizations](https://www.tensorflow.org/guide/data_performance) with loading data. Since no transformations were performed during loading dataset (hence doesn't make sense to do parallel calls), and `prefetch` was already applied with `AUTOTUNE`, the only viable thing left was `cache`. 

Although the % of `Dataset` in `tensorflow_stats` was reduced, no improvement was observed as the average time per step increased by 1.6ms and extended 0.2ms to input time (possibly insignificant). Additionally, `Op Time Spent on Eager Execution` increased by around 18% on `Host`, which was undesirable. 

No improvement was observed possibly because at least in this scenario the input loading has already been quite optimized already with `prefetch`. For the `TF_GPU_THREAD_MODE` tag, I am unsure why no enhancement on `Kernel launch time` was observed. This was also [observed by other people](https://github.com/tensorflow/profiler/issues/8) was well. 

#### Try 2 - **`ColabMinCast`**: Minimize `cast` ops

Another thing I tried was to look for alternatives for `cast` operations done in the model and function definitions. See below for an example. The changes are in `src/model_mincast.py`. 

In [ ]:
# this is just one example where trying to replace `cast` op achieves some speedup
a = tf.random.normal([10000,100])
%timeit -n 100 tf.cast(tf.math.equal(a, 0), tf.float32) # in the original code `src/model.py`
%timeit -n 100 tf.where(tf.math.equal(a, 0), 1.0, 0.0)  # changed in `src/model_mincast.py`

100 loops, best of 5: 127 µs per loop
100 loops, best of 5: 87.3 µs per loop


Interesting, this reduced `Kernel launch time` by more than 1ms (though unclear how significant this is) and I do not really have much explanation for this (as I also changed a couple of other things in the code like using `Sequential` to merge a few layers together but I doubt there was much gain from such change). Although this interestingly also reduced `Op Time Spent on Eager Execution - Host` from 34.6% to 5.5% (maybe the explicit casting relied more on the host eager execution), no improvement from the baseline was achieved, as `Host Compute Time` increased instead. 

Minimizing the `cast` operation did not gain any speedup, potentially because this particular operation was not really present or took much time in the GPU, was it didn't appear in the pie chart. 

#### Try 3 - **`ColabXLA`**: Turn on XLA flags

Lastly, following suggestion from <https://www.tensorflow.org/guide/gpu_performance_analysis> I tried to do `mixed_precision` but that didn't work as the loss (even with the suggested scaled loss and unscaled gradients) was not calculated correctly (plus Colab warned that this GPU was not optimized for it). So I tried turning the XLA tag instead and do explicit compiling for the `scaled_dot_product_attention`, as the [documentation](https://www.tensorflow.org/xla) said that sped up BERT models. 

``` python
os.environ['TF_XLA_FLAGS']='--tf_xla_auto_jit=2'

@tf.function(jit_compile=True)
def scaled_dot_product_attention(queries, keys, values, mask):
    product = tf.matmul(queries, keys, transpose_b=True)
    scaled_product = product / tf.math.sqrt(tf.cast(tf.shape(keys)[-1], tf.float32))
    scaled_product += (mask * -1e9)
    return tf.matmul(tf.nn.softmax(scaled_product, axis=-1), values)
```

This increased `TF Op Placement - Device` signifcantly to 99.6% but simultaneously increased `Op Time Spent on Eager Execution - Host` pronouncedly to 83.7%. The average time per step was slightly higher but potentially not significantly. This did not lead to improvement, possibly because this GPU device was not optimized for it much or the operations in `scaled_dot_product_attention` did not fuse optimally. 

### 4. Run the `unimproved version` notebook on ThetaGPU single-gpu queue, report your finding on profiling result difference. Are those bottlenecks you identified still bottlenecks?

See the table and pie charts above for the results. 

The `Device compute time` was significantly improved as ThetaGPU (22.2ms) seems to be much more powerful than Colab (~149-150ms). `Host Compute Time` and `Input time`, though already small, were also more optimal on ThetaGPU than on Colab. `TF Op Placement` percentages were not as optimal as Colab but that doesn't matter much. The `Op Time Spent on Eager Execution` was significantly reduced for both `host` and `device` compared to all Colab versions. 

The only bottleneck that appears to be on ThetaGPU but not so much on Colab was the `Kernel launch time` (47ms for ThetaGPU vs 17-19ms for Colab). Maybe changing the flags might have ameliorated this. Regardless, since the compute time was significantly reduced, the total average timeper step was much reduced, by nerly 60%. 

In terms of operation statistics, interestingly `MatMul` and `Argmax` were among the top for ThetaGPU although `MatMul` and `Transpose` were the top 2 for Colab. This is possibly because ThetaGPU devices are optimized for expensive matrix multiplication operations, hence explaining why this supposed bottleneck on Colab (can't get around unless knowing how sparse the matrices are) is easily solved on ThetaGPU. 

# Source repo
<https://github.com/tuanpham96/DeepLearningSystems-Fall2021/tree/main/HW5>

# Initialization

In [ ]:
%%capture
!pip install -U tensorboard-plugin-profile

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/MyDrive/Courses/Fall 2021/dlsys/DeepLearningSystems-Fall2021/HW5"

Mounted at /content/drive
/content/drive/MyDrive/Courses/Fall 2021/dlsys/DeepLearningSystems-Fall2021/HW5


# Data preparations

See see the [data-setup.ipynb](https://github.com/tuanpham96/DeepLearningSystems-Fall2021/tree/main/HW5/notebooks/data-setup.ipynb) notebook

# ThetaGPU
See the [HW5-thetagpu.ipynb](https://github.com/tuanpham96/DeepLearningSystems-Fall2021/tree/main/HW5/notebooks/HW5-thetagpu.ipynb) notebook

# Colab

## Essentials

In [ ]:
from datetime import datetime
from src.routines import *

In [ ]:
# Define paths
DATA_PATH = 'data/eng_spa_translations'
OUTPUT_PATH = 'output'
TRAIN_FILENAME = 'spa.txt'
URL_NONBREAKING_FILES = ['nonbreaking_prefix.en', 'nonbreaking_prefix.es']

In [ ]:
# Define configs
data_files = configure_datafiles(
    data_path               = DATA_PATH, 
    train_filename          = TRAIN_FILENAME, 
    nonbreaking_filenames   = URL_NONBREAKING_FILES
)

model_config = dict(    
    d_model                 = 512,
    n_layers                = 4,
    FFN_units               = 512,
    n_heads                 = 8,
    dropout_rate            = 0.1,
    act_fun                 = 'relu',
)


## Baseline

In [ ]:
# Load and tranform data 
dataset, token_dset = load_datasets(data_files) 

# Clean the session
tf.keras.backend.clear_session()  

In [ ]:
# Model name 
model_name = 'transformer-ColabBaseline'
# Create model
transformer = Transformer(
    vocab_size_enc=token_dset['input']['num_words'], 
    vocab_size_dec=token_dset['target']['num_words'],
    **model_config
)
# Compile model 
compile_model(transformer, model_config)
# Fit with callbacks
fit_model_with_callbacks(transformer, dataset, model_name, num_epochs=2)

Epoch 1/2
1250/1250 [==============================] - 228s 172ms/step - loss: 2.5786 - train_accuracy: 0.1805
Epoch 2/2
1250/1250 [==============================] - 210s 168ms/step - loss: 1.5255 - train_accuracy: 0.2731


## Try 1: Set `TF_GPU_THREAD_MODE=gpu_private` and use `cache` for data


In [ ]:
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private' # Change flag 
dataset, token_dset = load_datasets(data_files, use_cache=True) # Use flag 
tf.keras.backend.clear_session()  

In [ ]:
# Model name 
model_name = 'transformer-ColabFlagNCache'
# Create model
transformer = Transformer(
    vocab_size_enc=token_dset['input']['num_words'], 
    vocab_size_dec=token_dset['target']['num_words'],
    **model_config
)
# Compile model 
compile_model(transformer, model_config)
# Fit with callbacks
fit_model_with_callbacks(transformer, dataset, model_name, num_epochs=2)

Epoch 1/2
1250/1250 [==============================] - 223s 171ms/step - loss: 2.5767 - train_accuracy: 0.1805
Epoch 2/2
1250/1250 [==============================] - 209s 167ms/step - loss: 1.5351 - train_accuracy: 0.2717


## Try 2: Minimize `cast` ops

In [ ]:
# this is just one example where trying to replace `cast` op achieves some speedup
a = tf.random.normal([10000,100])
%timeit -n 100 tf.cast(tf.math.equal(a, 0), tf.float32)
%timeit -n 100 tf.where(tf.math.equal(a, 0), 1.0, 0.0)

100 loops, best of 5: 127 µs per loop
100 loops, best of 5: 87.3 µs per loop


In [ ]:
from src.model_mincast import *

In [ ]:
# Model name 
model_name = 'transformer-ColabMinCast'
# Create model
transformer = Transformer(
    vocab_size_enc=token_dset['input']['num_words'], 
    vocab_size_dec=token_dset['target']['num_words'],
    **model_config
)
# Compile model 
compile_model(transformer, model_config)
# Fit with callbacks
fit_model_with_callbacks(transformer, dataset, model_name, num_epochs=2)

Epoch 1/2
1250/1250 [==============================] - 228s 173ms/step - loss: 2.5716 - train_accuracy: 0.1809
Epoch 2/2
1250/1250 [==============================] - 213s 170ms/step - loss: 1.5422 - train_accuracy: 0.2703


## Try 3: Turn on `XLA` flags

In [ ]:
os.environ['TF_XLA_FLAGS']='--tf_xla_auto_jit=2'

In [ ]:
# reload the baseline model definitions instead of the mincast version
from src.model import * 

In [ ]:
del scaled_dot_product_attention

In [ ]:
@tf.function(jit_compile=True)
def scaled_dot_product_attention(queries, keys, values, mask):
    product = tf.matmul(queries, keys, transpose_b=True)
    scaled_product = product / tf.math.sqrt(tf.cast(tf.shape(keys)[-1], tf.float32))
    scaled_product += (mask * -1e9)
    return tf.matmul(tf.nn.softmax(scaled_product, axis=-1), values)

In [ ]:
# Model name 
model_name = 'transformer-ColabXLA'
# Create model
transformer = Transformer(
    vocab_size_enc=token_dset['input']['num_words'], 
    vocab_size_dec=token_dset['target']['num_words'],
    **model_config
)
# Compile model 
compile_model(transformer, model_config)
# Fit with callbacks
fit_model_with_callbacks(transformer, dataset, model_name, num_epochs=2)

Epoch 1/2
1250/1250 [==============================] - 231s 175ms/step - loss: 2.5767 - train_accuracy: 0.1808
Epoch 2/2
1250/1250 [==============================] - 214s 171ms/step - loss: 1.5201 - train_accuracy: 0.2745


# Tensorboard

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# If needed to kill and reload
!ps aux | grep '[/]bin/tensorboard' | awk '{print $2}' | xargs kill
%reload_ext tensorboard

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
# in order to save with version control, not gonna save this as it makes this notebook 12M
# see the above summary instead
%tensorboard --logdir=logs